In [1]:
# ============================================
# ANÁLISIS PROFUNDO DE RESULTADOS
# ============================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Cargar datos
print("Cargando datos...")
summary = pd.read_csv('uno_agents_summary.csv')
detailed = pd.read_csv('uno_agents_detailed.csv')

print(f"✓ Resumen cargado: {len(summary)} agentes")
print(f"✓ Detallado cargado: {len(detailed)} registros\n")

# ============================================
# 1. RESUMEN GENERAL
# ============================================
print("="*70)
print("📊 RESUMEN GENERAL")
print("="*70)
display(summary)

# ============================================
# 2. ANÁLISIS DE RACHAS (victoria consecutivas)
# ============================================
print("\n" + "="*70)
print("🔥 ANÁLISIS DE RACHAS DE VICTORIAS")
print("="*70)

for agente in summary['Agente'].unique():
    datos_agente = detailed[detailed['Agente'] == agente]
    
    # Calcular rachas
    rachas = []
    racha_actual = 0
    for victoria in datos_agente['Victoria']:
        if victoria == 1:
            racha_actual += 1
        else:
            if racha_actual > 0:
                rachas.append(racha_actual)
            racha_actual = 0
    if racha_actual > 0:
        rachas.append(racha_actual)
    
    print(f"\n{agente}:")
    print(f"  Total partidas: {len(datos_agente)}")
    print(f"  Victorias: {datos_agente['Victoria'].sum()}")
    print(f"  Tasa victoria: {(datos_agente['Victoria'].mean()*100):.2f}%")
    
    if rachas:
        print(f"  🏆 Mejor racha: {max(rachas)} victorias consecutivas")
        print(f"  📈 Racha promedio: {np.mean(rachas):.2f}")
        print(f"  📊 Total de rachas: {len(rachas)}")
    else:
        print(f"  ⚠️ Sin rachas de victoria")

# ============================================
# 3. CONSISTENCIA EN EL TIEMPO
# ============================================
print("\n" + "="*70)
print("📉 ANÁLISIS DE CONSISTENCIA")
print("="*70)

for agente in summary['Agente'].unique():
    datos = detailed[detailed['Agente'] == agente].copy()
    
    # Dividir en bloques de 200 partidas
    num_bloques = len(datos) // 200
    if num_bloques > 0:
        datos['Bloque'] = (datos.reset_index(drop=True).index // 200)
        victorias_por_bloque = datos.groupby('Bloque')['Victoria'].mean() * 100
        
        print(f"\n{agente}:")
        print(f"  Bloques analizados: {num_bloques} (200 partidas c/u)")
        print(f"  Varianza entre bloques: {victorias_por_bloque.std():.2f}%")
        print(f"  Rango: {victorias_por_bloque.min():.1f}% - {victorias_por_bloque.max():.1f}%")
        
        if victorias_por_bloque.std() > 10:
            print(f"  ⚠️ ALTA VARIABILIDAD - Más dependiente de suerte")
        elif victorias_por_bloque.std() < 5:
            print(f"  ✅ BAJA VARIABILIDAD - Estrategia muy consistente")
        else:
            print(f"  ✓ VARIABILIDAD MODERADA")

# ============================================
# 4. VELOCIDAD DE JUEGO
# ============================================
print("\n" + "="*70)
print("⚡ ANÁLISIS DE VELOCIDAD")
print("="*70)

for agente in summary['Agente'].unique():
    datos = detailed[detailed['Agente'] == agente]
    
    print(f"\n{agente}:")
    print(f"  Tiempo promedio por jugada: {datos['Tiempo_Jugada_ms'].mean():.3f} ms")
    print(f"  Desviación estándar: {datos['Tiempo_Jugada_ms'].std():.3f} ms")
    print(f"  Turnos promedio por partida: {datos['Turnos_Totales'].mean():.2f}")
    
    # Partidas más rápidas vs más lentas
    partidas_ganadoras = datos[datos['Victoria'] == 1]
    partidas_perdedoras = datos[datos['Victoria'] == 0]
    
    if len(partidas_ganadoras) > 0 and len(partidas_perdedoras) > 0:
        turnos_gana = partidas_ganadoras['Turnos_Totales'].mean()
        turnos_pierde = partidas_perdedoras['Turnos_Totales'].mean()
        print(f"  Turnos cuando GANA: {turnos_gana:.2f}")
        print(f"  Turnos cuando PIERDE: {turnos_pierde:.2f}")
        
        if turnos_gana < turnos_pierde:
            print(f"  ✓ Gana más rápido que pierde (eficiente)")
        else:
            print(f"  ⚠️ Tarda más en ganar que en perder")

# ============================================
# 5. RANKING FINAL
# ============================================
print("\n" + "="*70)
print("🏆 RANKING FINAL")
print("="*70)

ranking = summary[['Agente', 'Tasa_Victoria_%', 'Victorias', 'Derrotas']].copy()
ranking = ranking.sort_values('Tasa_Victoria_%', ascending=False)
ranking['Posición'] = range(1, len(ranking) + 1)

print("\n")
for idx, row in ranking.iterrows():
    emoji = "🥇" if row['Posición'] == 1 else "🥈" if row['Posición'] == 2 else "🥉" if row['Posición'] == 3 else "  "
    print(f"{emoji} {row['Posición']}. {row['Agente']}: {row['Tasa_Victoria_%']:.2f}% ({row['Victorias']}W - {row['Derrotas']}L)")

# ============================================
# 6. DIFERENCIA ESTADÍSTICAMENTE SIGNIFICATIVA
# ============================================
print("\n" + "="*70)
print("📊 DIFERENCIAS ENTRE AGENTES")
print("="*70)

agentes = summary['Agente'].tolist()
mejor_agente = ranking.iloc[0]['Agente']
peor_agente = ranking.iloc[-1]['Agente']

diff_mejor_peor = ranking.iloc[0]['Tasa_Victoria_%'] - ranking.iloc[-1]['Tasa_Victoria_%']

print(f"\nMejor: {mejor_agente} ({ranking.iloc[0]['Tasa_Victoria_%']:.2f}%)")
print(f"Peor: {peor_agente} ({ranking.iloc[-1]['Tasa_Victoria_%']:.2f}%)")
print(f"Diferencia: {diff_mejor_peor:.2f} puntos porcentuales")

if diff_mejor_peor < 5:
    print("\n⚠️ CONCLUSIÓN: Diferencias MUY PEQUEÑAS (<5%)")
    print("   Esto sugiere que:")
    print("   - UNO tiene mucho factor de suerte (aleatorización de cartas)")
    print("   - Los agentes son muy similares en estrategia")
    print("   - Se necesitan estrategias MÁS diferenciadas")
elif diff_mejor_peor < 10:
    print("\n✓ CONCLUSIÓN: Diferencias MODERADAS (5-10%)")
    print("   Hay cierta diferencia pero no muy pronunciada")
elif diff_mejor_peor >= 10:
    print("\n✅ CONCLUSIÓN: Diferencias CLARAS (>10%)")
    print("   Los agentes tienen estrategias claramente diferenciadas")

print("\n" + "="*70)

Cargando datos...
✓ Resumen cargado: 4 agentes
✓ Detallado cargado: 8000 registros

📊 RESUMEN GENERAL


,Agente,Tasa_Victoria_%,Partidas_Jugadas,Victorias,Derrotas,Tiempo_Promedio_Jugada_ms,Turnos_Promedio_Por_Juego,Desv_Est_Tiempo_ms
0,CFR,55.823293,249,139,110,0.073438,91.421687,0.060222
1,Reglas,51.405622,249,128,121,0.060316,94.425703,0.055558
2,Probabilistico,46.987952,249,117,132,0.061141,99.220884,0.058578
3,Random,45.783133,249,114,135,0.060354,95.236948,0.058657



🔥 ANÁLISIS DE RACHAS DE VICTORIAS

CFR:
  Total partidas: 2000
  Victorias: 1237
  Tasa victoria: 61.85%
  🏆 Mejor racha: 16 victorias consecutivas
  📈 Racha promedio: 2.59
  📊 Total de rachas: 477

Reglas:
  Total partidas: 2000
  Victorias: 1113
  Tasa victoria: 55.65%
  🏆 Mejor racha: 9 victorias consecutivas
  📈 Racha promedio: 2.24
  📊 Total de rachas: 497

Probabilistico:
  Total partidas: 2000
  Victorias: 773
  Tasa victoria: 38.65%
  🏆 Mejor racha: 9 victorias consecutivas
  📈 Racha promedio: 1.59
  📊 Total de rachas: 486

Random:
  Total partidas: 2000
  Victorias: 314
  Tasa victoria: 15.70%
  🏆 Mejor racha: 4 victorias consecutivas
  📈 Racha promedio: 1.18
  📊 Total de rachas: 266

📉 ANÁLISIS DE CONSISTENCIA

CFR:
  Bloques analizados: 10 (200 partidas c/u)
  Varianza entre bloques: 3.11%
  Rango: 54.0% - 65.0%
  ✅ BAJA VARIABILIDAD - Estrategia muy consistente

Reglas:
  Bloques analizados: 10 (200 partidas c/u)
  Varianza entre bloques: 3.45%
  Rango: 51.0% - 60.5%
  ✅ B